In [32]:
import pandas as pd
import numpy as np
import tensorflow
import theano
import pickle

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals.joblib import parallel_backend

In [2]:
df = pd.read_csv('../data/df_for_model.csv')
# df['start_time'] = df['start_time']//1000
df = df[df['result'] != 0.5]
numeric_predictors = ['color', 'diff', 'game_time', 'start_time', 'weekday']
X = df[numeric_predictors]
y = np.array(df['result'])

In [3]:
X.shape

(2076, 5)

In [4]:
y.shape

(2076,)

In [5]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=6)

In [6]:
X_train = X[:1876]
y_train = y[:1876]
X_test = X[1876:]
y_test = y[1876:]

In [7]:
X_train.shape

(1876, 5)

In [8]:
X_test.shape

(200, 5)

In [9]:
y_train.shape

(1876,)

In [10]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [11]:
std_sclr.scale_

array([  0.49989999,  76.59092228, 131.5902732 ,   4.6553088 ,
         1.96720614])

In [63]:
def _classifier():
    classifier = Sequential()

    classifier.add(Dense(units=13, kernel_initializer='uniform',
                         activation='relu', input_dim=5))

    classifier.add(
        Dense(units=83, kernel_initializer='uniform', activation='softmax'))

    classifier.add(
        Dense(units=13, kernel_initializer='uniform', activation='relu'))

#     classifier.add(
#         Dense(units=109, kernel_initializer='uniform', activation='softmax'))

#     classifier.add(
#         Dense(units=47, kernel_initializer='uniform', activation='relu'))

#     classifier.add(
#         Dense(units=11, kernel_initializer='uniform', activation='softmax'))

    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['binary_accuracy'])

    return classifier

In [64]:
classifier = _classifier()

In [65]:
batch_list = [134, 67, 28, 14, 7, 4]
for num in batch_list:
    classifier.fit(X_train, y_train, batch_size=num, 
                   epochs=100, shuffle=False)

Epoch 1/100
1876/1876 [==============================] - 1s 556us/step - loss: 0.6932 - binary_accuracy: 0.4936
Epoch 2/100
1876/1876 [==============================] - 0s 41us/step - loss: 0.6932 - binary_accuracy: 0.5053
Epoch 3/100
1876/1876 [==============================] - 0s 44us/step - loss: 0.6931 - binary_accuracy: 0.5053
Epoch 4/100
1876/1876 [==============================] - 0s 38us/step - loss: 0.6931 - binary_accuracy: 0.5053
Epoch 5/100
1876/1876 [==============================] - 0s 47us/step - loss: 0.6931 - binary_accuracy: 0.5053
Epoch 6/100
1876/1876 [==============================] - 0s 47us/step - loss: 0.6931 - binary_accuracy: 0.5053
Epoch 7/100
1876/1876 [==============================] - 0s 35us/step - loss: 0.6929 - binary_accuracy: 0.5053
Epoch 8/100
1876/1876 [==============================] - 0s 36us/step - loss: 0.6898 - binary_accuracy: 0.5064
Epoch 9/100
1876/1876 [==============================] - 0s 31us/step - loss: 0.6773 - binary_accuracy: 0.6370


1876/1876 [==============================] - 0s 54us/step - loss: 0.5516 - binary_accuracy: 0.7074
Epoch 75/100
1876/1876 [==============================] - 0s 31us/step - loss: 0.5515 - binary_accuracy: 0.7068
Epoch 76/100
1876/1876 [==============================] - 0s 36us/step - loss: 0.5515 - binary_accuracy: 0.7068
Epoch 77/100
1876/1876 [==============================] - 0s 38us/step - loss: 0.5514 - binary_accuracy: 0.7063
Epoch 78/100
1876/1876 [==============================] - 0s 27us/step - loss: 0.5514 - binary_accuracy: 0.7063
Epoch 79/100
1876/1876 [==============================] - 0s 51us/step - loss: 0.5513 - binary_accuracy: 0.7063
Epoch 80/100
1876/1876 [==============================] - 0s 42us/step - loss: 0.5513 - binary_accuracy: 0.7063
Epoch 81/100
1876/1876 [==============================] - 0s 38us/step - loss: 0.5512 - binary_accuracy: 0.7058
Epoch 82/100
1876/1876 [==============================] - 0s 43us/step - loss: 0.5512 - binary_accuracy: 0.7068
Epoch

1876/1876 [==============================] - 0s 72us/step - loss: 0.5467 - binary_accuracy: 0.7084
Epoch 48/100
1876/1876 [==============================] - 0s 81us/step - loss: 0.5467 - binary_accuracy: 0.7084
Epoch 49/100
1876/1876 [==============================] - 0s 72us/step - loss: 0.5466 - binary_accuracy: 0.7079
Epoch 50/100
1876/1876 [==============================] - 0s 72us/step - loss: 0.5466 - binary_accuracy: 0.7079
Epoch 51/100
1876/1876 [==============================] - 0s 82us/step - loss: 0.5465 - binary_accuracy: 0.7079
Epoch 52/100
1876/1876 [==============================] - 0s 74us/step - loss: 0.5465 - binary_accuracy: 0.7090
Epoch 53/100
1876/1876 [==============================] - 0s 63us/step - loss: 0.5464 - binary_accuracy: 0.7084
Epoch 54/100
1876/1876 [==============================] - 0s 87us/step - loss: 0.5464 - binary_accuracy: 0.7079
Epoch 55/100
1876/1876 [==============================] - 0s 70us/step - loss: 0.5463 - binary_accuracy: 0.7084
Epoch

1876/1876 [==============================] - 0s 113us/step - loss: 0.5416 - binary_accuracy: 0.7122
Epoch 21/100
1876/1876 [==============================] - 0s 108us/step - loss: 0.5414 - binary_accuracy: 0.7127
Epoch 22/100
1876/1876 [==============================] - 0s 121us/step - loss: 0.5412 - binary_accuracy: 0.7122
Epoch 23/100
1876/1876 [==============================] - 0s 113us/step - loss: 0.5410 - binary_accuracy: 0.7148
Epoch 24/100
1876/1876 [==============================] - 0s 117us/step - loss: 0.5408 - binary_accuracy: 0.7138
Epoch 25/100
1876/1876 [==============================] - 0s 113us/step - loss: 0.5407 - binary_accuracy: 0.7154
Epoch 26/100
1876/1876 [==============================] - 0s 107us/step - loss: 0.5405 - binary_accuracy: 0.7148
Epoch 27/100
1876/1876 [==============================] - 0s 111us/step - loss: 0.5404 - binary_accuracy: 0.7164
Epoch 28/100
1876/1876 [==============================] - 0s 111us/step - loss: 0.5402 - binary_accuracy: 0.7

1876/1876 [==============================] - 0s 143us/step - loss: 0.5324 - binary_accuracy: 0.7159
Epoch 93/100
1876/1876 [==============================] - 0s 112us/step - loss: 0.5322 - binary_accuracy: 0.7148
Epoch 94/100
1876/1876 [==============================] - 0s 111us/step - loss: 0.5321 - binary_accuracy: 0.7148
Epoch 95/100
1876/1876 [==============================] - 0s 115us/step - loss: 0.5320 - binary_accuracy: 0.7148
Epoch 96/100
1876/1876 [==============================] - 0s 110us/step - loss: 0.5319 - binary_accuracy: 0.7154
Epoch 97/100
1876/1876 [==============================] - 0s 109us/step - loss: 0.5318 - binary_accuracy: 0.7148
Epoch 98/100
1876/1876 [==============================] - 0s 115us/step - loss: 0.5317 - binary_accuracy: 0.7170
Epoch 99/100
1876/1876 [==============================] - 0s 109us/step - loss: 0.5316 - binary_accuracy: 0.7148
Epoch 100/100
1876/1876 [==============================] - 0s 105us/step - loss: 0.5315 - binary_accuracy: 0.

1876/1876 [==============================] - 0s 218us/step - loss: 0.5274 - binary_accuracy: 0.7207
Epoch 65/100
1876/1876 [==============================] - 0s 231us/step - loss: 0.5273 - binary_accuracy: 0.7196
Epoch 66/100
1876/1876 [==============================] - 0s 229us/step - loss: 0.5272 - binary_accuracy: 0.7196
Epoch 67/100
1876/1876 [==============================] - 0s 216us/step - loss: 0.5271 - binary_accuracy: 0.7186
Epoch 68/100
1876/1876 [==============================] - 0s 228us/step - loss: 0.5270 - binary_accuracy: 0.7196
Epoch 69/100
1876/1876 [==============================] - 0s 222us/step - loss: 0.5269 - binary_accuracy: 0.7201
Epoch 70/100
1876/1876 [==============================] - 0s 213us/step - loss: 0.5268 - binary_accuracy: 0.7186
Epoch 71/100
1876/1876 [==============================] - 0s 222us/step - loss: 0.5268 - binary_accuracy: 0.7196
Epoch 72/100
1876/1876 [==============================] - 0s 207us/step - loss: 0.5267 - binary_accuracy: 0.7

1876/1876 [==============================] - 1s 415us/step - loss: 0.5209 - binary_accuracy: 0.7329
Epoch 37/100
1876/1876 [==============================] - 1s 407us/step - loss: 0.5206 - binary_accuracy: 0.7345
Epoch 38/100
1876/1876 [==============================] - 1s 401us/step - loss: 0.5204 - binary_accuracy: 0.7345
Epoch 39/100
1876/1876 [==============================] - 1s 401us/step - loss: 0.5202 - binary_accuracy: 0.7367
Epoch 40/100
1876/1876 [==============================] - 1s 410us/step - loss: 0.5200 - binary_accuracy: 0.7351
Epoch 41/100
1876/1876 [==============================] - 1s 404us/step - loss: 0.5197 - binary_accuracy: 0.7361
Epoch 42/100
1876/1876 [==============================] - 1s 403us/step - loss: 0.5195 - binary_accuracy: 0.7372
Epoch 43/100
1876/1876 [==============================] - 1s 408us/step - loss: 0.5193 - binary_accuracy: 0.7372
Epoch 44/100
1876/1876 [==============================] - 1s 411us/step - loss: 0.5190 - binary_accuracy: 0.7

1876/1876 [==============================] - 1s 755us/step - loss: 0.5047 - binary_accuracy: 0.7484
Epoch 9/100
1876/1876 [==============================] - 1s 799us/step - loss: 0.5040 - binary_accuracy: 0.7484
Epoch 10/100
1876/1876 [==============================] - 2s 837us/step - loss: 0.5039 - binary_accuracy: 0.7527
Epoch 11/100
1876/1876 [==============================] - 2s 973us/step - loss: 0.5033 - binary_accuracy: 0.7543
Epoch 12/100
1876/1876 [==============================] - 1s 705us/step - loss: 0.5031 - binary_accuracy: 0.7532
Epoch 13/100
1876/1876 [==============================] - 1s 727us/step - loss: 0.5023 - binary_accuracy: 0.7564
Epoch 14/100
1876/1876 [==============================] - 1s 745us/step - loss: 0.5020 - binary_accuracy: 0.7543
Epoch 15/100
1876/1876 [==============================] - 1s 748us/step - loss: 0.5014 - binary_accuracy: 0.7553
Epoch 16/100
1876/1876 [==============================] - 1s 749us/step - loss: 0.5012 - binary_accuracy: 0.75

1876/1876 [==============================] - 1s 685us/step - loss: 0.4743 - binary_accuracy: 0.7671
Epoch 81/100
1876/1876 [==============================] - 1s 682us/step - loss: 0.4727 - binary_accuracy: 0.7665
Epoch 82/100
1876/1876 [==============================] - 1s 682us/step - loss: 0.4729 - binary_accuracy: 0.7681
Epoch 83/100
1876/1876 [==============================] - 1s 684us/step - loss: 0.4728 - binary_accuracy: 0.7676
Epoch 84/100
1876/1876 [==============================] - 1s 679us/step - loss: 0.4776 - binary_accuracy: 0.7660
Epoch 85/100
1876/1876 [==============================] - 1s 688us/step - loss: 0.4705 - binary_accuracy: 0.7697
Epoch 86/100
1876/1876 [==============================] - 1s 736us/step - loss: 0.4699 - binary_accuracy: 0.7697
Epoch 87/100
1876/1876 [==============================] - 2s 869us/step - loss: 0.4696 - binary_accuracy: 0.7719
Epoch 88/100
1876/1876 [==============================] - 2s 1ms/step - loss: 0.4682 - binary_accuracy: 0.771

In [66]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [67]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[73, 28],
       [33, 66]])

In [68]:
print(f'Accuracy = {(cm[0][0]+cm[1][1])/(cm.sum())/.01}%')

Accuracy = 69.5%


### 75.5% Accuracy on 1824 training, 200 test
__3 layers__
- layer 1 = ReLU X 7
- layer 2 = SoftMax X 14
- layer 3 = ReLU X 7
- output layer = Sigmoid X 1

__metrics__ = Cosine

__loss__ = binary_crossentropy

__Optimizer__ = Nadam

__Batch size__ = 12

__Epochs__ = 175

no __diff__

In [16]:
classifier = KerasClassifier(build_fn=_classifier, batch_size=12, epochs=200)

In [17]:
accus = cross_val_score(estimator=classifier, X=X_train,
                        y=y_train, cv=10, n_jobs=-2, scoring='accuracy')
accus

Epoch 1/200
1688/1688 [==============================] - 1s 532us/step - loss: 0.6931 - acc: 0.5148
Epoch 2/200
1688/1688 [==============================] - 0s 222us/step - loss: 0.6359 - acc: 0.6742
Epoch 3/200
1688/1688 [==============================] - 0s 216us/step - loss: 0.5818 - acc: 0.7062
Epoch 4/200
1688/1688 [==============================] - 0s 222us/step - loss: 0.5709 - acc: 0.7020
Epoch 5/200
1688/1688 [==============================] - 0s 224us/step - loss: 0.5660 - acc: 0.7026
Epoch 6/200
1688/1688 [==============================] - 0s 235us/step - loss: 0.5627 - acc: 0.7056
Epoch 7/200
1688/1688 [==============================] - 0s 229us/step - loss: 0.5603 - acc: 0.6985
Epoch 8/200
1688/1688 [==============================] - 0s 227us/step - loss: 0.5589 - acc: 0.7038
Epoch 9/200
1688/1688 [==============================] - 0s 220us/step - loss: 0.5569 - acc: 0.7133
Epoch 10/200
1688/1688 [==============================] - 0s 240us/step - loss: 0.5570 - acc: 0.7050

1688/1688 [==============================] - 0s 233us/step - loss: 0.5423 - acc: 0.7115
Epoch 83/200
1688/1688 [==============================] - 0s 220us/step - loss: 0.5436 - acc: 0.7115
Epoch 84/200
1688/1688 [==============================] - 0s 230us/step - loss: 0.5427 - acc: 0.7162
Epoch 85/200
1688/1688 [==============================] - 0s 227us/step - loss: 0.5407 - acc: 0.7180
Epoch 86/200
1688/1688 [==============================] - 0s 212us/step - loss: 0.5422 - acc: 0.7109
Epoch 87/200
1688/1688 [==============================] - 0s 228us/step - loss: 0.5417 - acc: 0.7145
Epoch 88/200
1688/1688 [==============================] - 0s 220us/step - loss: 0.5406 - acc: 0.7162
Epoch 89/200
1688/1688 [==============================] - 0s 226us/step - loss: 0.5425 - acc: 0.7156
Epoch 90/200
1688/1688 [==============================] - 0s 216us/step - loss: 0.5416 - acc: 0.7127
Epoch 91/200
1688/1688 [==============================] - 0s 244us/step - loss: 0.5409 - acc: 0.7150
Epo

1688/1688 [==============================] - 0s 223us/step - loss: 0.5342 - acc: 0.7210
Epoch 163/200
1688/1688 [==============================] - 0s 222us/step - loss: 0.5354 - acc: 0.7091
Epoch 164/200
1688/1688 [==============================] - 0s 219us/step - loss: 0.5335 - acc: 0.7227
Epoch 165/200
1688/1688 [==============================] - 0s 219us/step - loss: 0.5334 - acc: 0.7133
Epoch 166/200
1688/1688 [==============================] - 0s 223us/step - loss: 0.5328 - acc: 0.7192
Epoch 167/200
1688/1688 [==============================] - 0s 241us/step - loss: 0.5337 - acc: 0.7210
Epoch 168/200
1688/1688 [==============================] - 0s 228us/step - loss: 0.5325 - acc: 0.7245
Epoch 169/200
1688/1688 [==============================] - 0s 216us/step - loss: 0.5338 - acc: 0.7210
Epoch 170/200
1688/1688 [==============================] - 0s 213us/step - loss: 0.5333 - acc: 0.7121
Epoch 171/200
1688/1688 [==============================] - 0s 226us/step - loss: 0.5336 - acc: 0

Epoch 43/200
1688/1688 [==============================] - 1s 298us/step - loss: 0.6932 - acc: 0.5071
Epoch 44/200
1688/1688 [==============================] - 0s 259us/step - loss: 0.6932 - acc: 0.5071
Epoch 45/200
1688/1688 [==============================] - 0s 244us/step - loss: 0.6931 - acc: 0.5071
Epoch 46/200
1688/1688 [==============================] - 0s 254us/step - loss: 0.6932 - acc: 0.5071
Epoch 47/200
1688/1688 [==============================] - 0s 241us/step - loss: 0.6932 - acc: 0.5071
Epoch 48/200
1688/1688 [==============================] - 0s 242us/step - loss: 0.6932 - acc: 0.5071
Epoch 49/200
1688/1688 [==============================] - 0s 246us/step - loss: 0.6932 - acc: 0.5071
Epoch 50/200
1688/1688 [==============================] - 0s 239us/step - loss: 0.6932 - acc: 0.5071
Epoch 51/200
1688/1688 [==============================] - 0s 251us/step - loss: 0.6932 - acc: 0.5071
Epoch 52/200
1688/1688 [==============================] - 0s 258us/step - loss: 0.6932 - ac

1688/1688 [==============================] - 0s 227us/step - loss: 0.6932 - acc: 0.5071
Epoch 124/200
1688/1688 [==============================] - 0s 223us/step - loss: 0.6931 - acc: 0.5071
Epoch 125/200
1688/1688 [==============================] - 0s 211us/step - loss: 0.6931 - acc: 0.5071
Epoch 126/200
1688/1688 [==============================] - 0s 226us/step - loss: 0.6932 - acc: 0.5071
Epoch 127/200
1688/1688 [==============================] - 0s 212us/step - loss: 0.6932 - acc: 0.5071
Epoch 128/200
1688/1688 [==============================] - 0s 214us/step - loss: 0.6932 - acc: 0.5071
Epoch 129/200
1688/1688 [==============================] - 0s 231us/step - loss: 0.6931 - acc: 0.5071
Epoch 130/200
1688/1688 [==============================] - 0s 226us/step - loss: 0.6931 - acc: 0.5071
Epoch 131/200
1688/1688 [==============================] - 0s 217us/step - loss: 0.6932 - acc: 0.5071
Epoch 132/200
1688/1688 [==============================] - 0s 248us/step - loss: 0.6932 - acc: 0

...

1689/1689 [==============================] - 0s 224us/step - loss: 0.6932 - acc: 0.5086
Epoch 92/200
1689/1689 [==============================] - 0s 246us/step - loss: 0.6931 - acc: 0.5086
Epoch 93/200
1689/1689 [==============================] - 0s 243us/step - loss: 0.6932 - acc: 0.5086
Epoch 94/200
1689/1689 [==============================] - 0s 278us/step - loss: 0.6931 - acc: 0.5086
Epoch 95/200
1689/1689 [==============================] - 0s 291us/step - loss: 0.6932 - acc: 0.5086
Epoch 96/200
1689/1689 [==============================] - 1s 300us/step - loss: 0.6931 - acc: 0.5086
Epoch 97/200
1689/1689 [==============================] - 0s 255us/step - loss: 0.6931 - acc: 0.5086
Epoch 98/200
1689/1689 [==============================] - 0s 233us/step - loss: 0.6931 - acc: 0.5086
Epoch 99/200
1689/1689 [==============================] - 0s 217us/step - loss: 0.6931 - acc: 0.5086
Epoch 100/200
1689/1689 [==============================] - 0s 229us/step - loss: 0.6931 - acc: 0.5086
Ep

1689/1689 [==============================] - 0s 246us/step - loss: 0.6931 - acc: 0.5086
Epoch 172/200
1689/1689 [==============================] - 0s 235us/step - loss: 0.6931 - acc: 0.5086
Epoch 173/200
1689/1689 [==============================] - 0s 231us/step - loss: 0.6931 - acc: 0.5086
Epoch 174/200
1689/1689 [==============================] - 1s 322us/step - loss: 0.6931 - acc: 0.5086
Epoch 175/200
1689/1689 [==============================] - 0s 254us/step - loss: 0.6931 - acc: 0.5086
Epoch 176/200
1689/1689 [==============================] - 0s 248us/step - loss: 0.6931 - acc: 0.5086
Epoch 177/200
1689/1689 [==============================] - 0s 233us/step - loss: 0.6931 - acc: 0.5086
Epoch 178/200
1689/1689 [==============================] - 0s 221us/step - loss: 0.6931 - acc: 0.5086
Epoch 179/200
1689/1689 [==============================] - 0s 246us/step - loss: 0.6931 - acc: 0.5086
Epoch 180/200
1689/1689 [==============================] - 0s 220us/step - loss: 0.6932 - acc: 0

1689/1689 [==============================] - 0s 248us/step - loss: 0.6932 - acc: 0.5050
Epoch 53/200
1689/1689 [==============================] - 0s 225us/step - loss: 0.6932 - acc: 0.5050
Epoch 54/200
1689/1689 [==============================] - 0s 225us/step - loss: 0.6933 - acc: 0.5050
Epoch 55/200
1689/1689 [==============================] - 0s 240us/step - loss: 0.6932 - acc: 0.5050
Epoch 56/200
1689/1689 [==============================] - 0s 239us/step - loss: 0.6932 - acc: 0.4932
Epoch 57/200
1689/1689 [==============================] - 0s 237us/step - loss: 0.6933 - acc: 0.5050
Epoch 58/200
1689/1689 [==============================] - 0s 227us/step - loss: 0.6932 - acc: 0.5050
Epoch 59/200
1689/1689 [==============================] - 0s 244us/step - loss: 0.6932 - acc: 0.5050
Epoch 60/200
1689/1689 [==============================] - 1s 297us/step - loss: 0.6932 - acc: 0.5050
Epoch 61/200
1689/1689 [==============================] - 0s 241us/step - loss: 0.6932 - acc: 0.5050
Epo

1689/1689 [==============================] - 0s 288us/step - loss: 0.6932 - acc: 0.4956
Epoch 133/200
1689/1689 [==============================] - 0s 241us/step - loss: 0.6932 - acc: 0.5050
Epoch 134/200
1689/1689 [==============================] - 0s 271us/step - loss: 0.6932 - acc: 0.5050
Epoch 135/200
1689/1689 [==============================] - 0s 227us/step - loss: 0.6932 - acc: 0.5050
Epoch 136/200
1689/1689 [==============================] - 0s 289us/step - loss: 0.6932 - acc: 0.5050
Epoch 137/200
1689/1689 [==============================] - 0s 234us/step - loss: 0.6932 - acc: 0.5050
Epoch 138/200
1689/1689 [==============================] - 0s 230us/step - loss: 0.6932 - acc: 0.5050
Epoch 139/200
1689/1689 [==============================] - 0s 232us/step - loss: 0.6932 - acc: 0.5050
Epoch 140/200
1689/1689 [==============================] - 0s 284us/step - loss: 0.6931 - acc: 0.5050
Epoch 141/200
1689/1689 [==============================] - 1s 307us/step - loss: 0.6932 - acc: 0

array([0.70212766, 0.4893617 , 0.65425532, 0.5106383 , 0.66489362,
       0.4787234 , 0.51336898, 0.52941176, 0.47593583, 0.50802139])

In [19]:
accu_mean = accus.mean()
accu_mean

0.5526737967914438

In [20]:
accu_var = accus.std()
accu_var

0.08152371401080122

In [21]:
accus

array([0.70212766, 0.4893617 , 0.65425532, 0.5106383 , 0.66489362,
       0.4787234 , 0.51336898, 0.52941176, 0.47593583, 0.50802139])

In [70]:
def dropout_classifier(optimizer, metrics):
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=40, kernel_initializer='uniform', activation='softsign'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics])
    return classifier

In [71]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [72]:
parameters = {'batch_size': [4, 8, 12, 16, 32, 64, 128],
              'nb_epoch': [50, 100, 250, 500, 1000],
              'optimizer': ['Nadam', 'Adadelta', 'Adamax', 'SGD', 'Adam', 'RMSprop'],
              'metrics': ['accuracy', 'mean_absolute_error', 'squared_hinge', 'binary_accuracy', 'mean_absolute_percentage_error']
              }

In [73]:
grid_search = GridSearchCV(classifier, parameters,
                           scoring='accuracy', cv=10, n_jobs=-4)

In [74]:
grid_search = grid_search.fit(X=X_train, y=y_train)

Epoch 1/1
1688/1688 [==============================] - 3s 1ms/step - loss: 0.6351 - acc: 0.6149
Epoch 1/1
1688/1688 [==============================] - 3s 2ms/step - loss: 0.6313 - acc: 0.6481
Epoch 1/1
1688/1688 [==============================] - 3s 2ms/step - loss: 0.6933 - acc: 0.4953
Epoch 1/1
1688/1688 [==============================] - 3s 2ms/step - loss: 0.6152 - acc: 0.6475
Epoch 1/1
1688/1688 [==============================] - 3s 2ms/step - loss: 0.6195 - acc: 0.6333
Epoch 1/1
1688/1688 [==============================] - 3s 2ms/step - loss: 0.6212 - acc: 0.6493
Epoch 1/1
1689/1689 [==============================] - 3s 2ms/step - loss: 0.6246 - acc: 0.6442
Epoch 1/1
1689/1689 [==============================] - 3s 2ms/step - loss: 0.6327 - acc: 0.6347
Epoch 1/1
1048/1689 [=================>............] - ETA: 1s - loss: 0.6422 - acc: 0.6231

KeyboardInterrupt: 

In [ ]:
best_params = grid_search.best_params_
best_params

In [ ]:
best_accu = grid_search.best_score_
best_accu

In [ ]:
y_pred = grid_search.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
(56+38)/(67+57)